In [14]:
# the cnn module provides classes for training/predicting with various types of CNNs
import opensoundscape

#other utilities and packages
import os
import torch
from pathlib import Path
import numpy as np
import pandas as pd
import random
import subprocess
from glob import glob
import sklearn

#set up plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'


In [15]:
#Create metadata CSV for OpenSoundScape module


data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

filepath_presence_dict = {"filepath":[], "presence":[]}

weto_train_dir = os.path.join(data_dir, 'weto', 'train')
for set_key in ['positive', 'negative']:
    set_dir = os.path.join(weto_train_dir, set_key)
    filenames = os.listdir(set_dir)
    filepath_list = filepath_presence_dict["filepath"]
    presence_list = filepath_presence_dict["presence"]
    for name in filenames:
        filepath = os.path.join(set_dir, name)
        filepath_list.append(filepath)
    filepath_presence_dict.update({'filepath': filepath_list})
    if set_key == "positive":
        presence_list = list(np.repeat("positive", len(filenames)))
        filepath_presence_dict.update({'presence': presence_list})
    if set_key == "negative":
        presence_list.extend(list(np.repeat("negative", len(filenames))))
        filepath_presence_dict.update({'presence': presence_list})    


In [16]:
meta_weto = pd.get_dummies(pd.DataFrame(filepath_presence_dict), columns = ['presence'], prefix = "", prefix_sep="", dtype = int).set_index('filepath')
display(meta_weto.head(3), meta_weto.tail(3))
print(f"There are {np.sum(meta_weto['positive'])} positive samples")
print(f"There are {np.sum(meta_weto['negative'])} negative samples")

,negative,positive
filepath,,
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\positive\A-11-E_20210430_230000_99_5_0-3.wav,0,1
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\positive\A-11-E_20210503_230000_26_9_0-3.wav,0,1
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\positive\A-11-E_20210503_230000_26_9_3-6.wav,0,1


,negative,positive
filepath,,
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\negative\A-9_20210515_230000_120_8_3-6.wav,1,0
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\negative\A-9_20210515_230000_60_4_0-3.wav,1,0
\\ban-files-02\Groups\EI Monitoring\Amphibian Acoustic Classifier\Analysis\data\weto\train\negative\A-9_20210515_230000_63_5_0-3.wav,1,0


There are 590 positive samples
There are 755 negative samples


In [17]:
meta_weto_balanced = opensoundscape.data_selection.resample(meta_weto, 
                                       n_samples_per_class = 800, 
                                       upsample=True, 
                                       downsample=True, 
                                       with_replace=False, 
                                       random_state=42)

print(f"There are {np.sum(meta_weto_balanced['positive'])} positive samples")
print(f"There are {np.sum(meta_weto_balanced['negative'])} negative samples")

There are 800 positive samples
There are 800 negative samples


In [18]:
train_df, valid_df = sklearn.model_selection.train_test_split(meta_weto_balanced, test_size=0.1, random_state=0)


In [19]:
# Create a CNN object designed to recognize 3-second samples
from opensoundscape import CNN

# Use resnet34 architecture
architecture = 'resnet34'

# Can use this code to get your classes, if needed
class_list = list(train_df.columns)

model = CNN(
    architecture = architecture,
    classes = class_list,
    sample_duration = 3) #3s, selected above

print(f'model.device is: {model.device}')

model.device is: cpu


In [20]:
model.device = 'gpu'
model.device

'gpu'

In [21]:
#Set up WandB
import wandb
try:
    wandb.login()
    wandb_session = wandb.init(
        entity='pca_bioacoustics', #replace with your entity/group name
        project='bioacoustics_model',
        name='Train CNN',
    )
except: #if wandb.init fails, don't use wandb logging
    print('failed to create wandb session. wandb session will be None')
    wandb_session = None

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
c:\Users\gavin hurd\.conda\envs\amphib_pt_remote\lib\site-packages\wandb\sdk\lib\ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  self._disabled = False


c:\Users\gavin hurd\.conda\envs\amphib_pt_remote\lib\site-packages\wandb\sdk\lib\ipython.py:89: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  self._disabled = True


In [22]:
checkpoint_folder = Path("model_training_checkpoints")
checkpoint_folder.mkdir(exist_ok=True)
checkpoint_folder_path = os.path.join(os.getcwd(), "model_training_checkpoints")

In [23]:
model.device = 'cpu'

In [24]:
#%%capture --no-stdout --no-display
# Uncomment the line above to silence outputs from this cell

model.train(
    train_df,
    valid_df,
    epochs = 100,
    batch_size = 32,
    log_interval = 100, #log progress every 100 batches
    #num_workers = 4, #4 parallelized cpu tasks for preprocessing
    wandb_session = wandb_session,
    save_interval = 10, #save checkpoint every 10 epochs
    save_path = checkpoint_folder_path #location to save checkpoints
)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\GAVINH~1\\AppData\\Local\\Temp\\tmpsy9dt8nw\\Samples / training samples.table.json'